In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
dog_data = pd.read_csv('NYC_Dog_Licensing_Dataset_20241105.csv')
dog_data.head()

C:\Users\cshar\AppData\Local\Temp\ipykernel_13176\421662572.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dog_data = pd.read_csv('NYC_Dog_Licensing_Dataset_20241105.csv')


,AnimalName,AnimalGender,AnimalBirthYear,BreedName,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
0,PAIGE,F,2014,American Pit Bull Mix / Pit Bull Mix,10035.0,09-12-2014,09-12-2017,2016
1,YOGI,M,2010,Boxer,10465.0,09-12-2014,10-02-2017,2016
2,ALI,M,2014,Basenji,10013.0,09-12-2014,09-12-2019,2016
3,QUEEN,F,2013,Akita Crossbreed,10013.0,09-12-2014,09-12-2017,2016
4,LOLA,F,2009,Maltese,10028.0,09-12-2014,10-09-2017,2016


In [19]:
#Data Cleaning
#fixing the warning Pandas gave us by specifying the AnimalBirth column as a string, we will later convert it
dog_data = pd.read_csv('NYC_Dog_Licensing_Dataset_20241105.csv', dtype={'AnimalBirthYear': 'str'}, low_memory=False)

#Converting the dates into Pandas Datetime format
dog_data['LicenseIssuedDate'] = pd.to_datetime(dog_data['LicenseIssuedDate'], errors='coerce')
dog_data['LicenseExpiredDate'] = pd.to_datetime(dog_data['LicenseExpiredDate'], errors='coerce')

#converting AnimalBirthYear to integer and setting any problematic value to NaN using 'coerce'
dog_data['AnimalBirthYear'] = pd.to_numeric(dog_data['AnimalBirthYear'], errors='coerce')

#exploring the total missing values in each column
print(dog_data.isnull().sum())

AnimalName              1387
AnimalGender              21
AnimalBirthYear           30
BreedName                  0
ZipCode                    9
LicenseIssuedDate     370485
LicenseExpiredDate    389919
Extract Year               0
dtype: int64


In [23]:
# I have chosen to drop the missing values
dog_data.dropna(subset=['LicenseIssuedDate', 'LicenseExpiredDate'], inplace=True)
dog_data.dropna(subset=['AnimalGender'], inplace=True)
dog_data.dropna(subset=['ZipCode'], inplace=True)

dog_data.isnull().sum()

AnimalName            498
AnimalGender            0
AnimalBirthYear         9
BreedName               0
ZipCode                 0
LicenseIssuedDate       0
LicenseExpiredDate      0
Extract Year            0
dtype: int64

In [25]:
#Converting the missing values with an 'unknown' value
dog_data['AnimalName'].fillna('Unknown', inplace=True)

# dropping the few birth year values
dog_data.dropna(subset=['AnimalBirthYear'], inplace=True)

dog_data.isnull().sum()

C:\Users\cshar\AppData\Local\Temp\ipykernel_13176\1515028959.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dog_data['AnimalName'].fillna('Unknown', inplace=True)


AnimalName            0
AnimalGender          0
AnimalBirthYear       0
BreedName             0
ZipCode               0
LicenseIssuedDate     0
LicenseExpiredDate    0
Extract Year          0
dtype: int64